Step 1: Setup and Library Imports

In [ ]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from groq import Groq  # Assuming you have access to Groq's client
import os

def load_json_data(json_path):
    with open(json_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def create_tfidf_vectors(data):
    documents = [entry['question'] + " " + entry['code'] for entry in data]  # Context includes both code and the question
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(documents)
    return vectorizer, tfidf_matrix
